In [71]:
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta

In [4]:
df_raw = pd.read_excel("MMSecondPricesOrders_Fall2019.xlsx")

### df & df_time

In [49]:
df = df_raw.iloc[:,0:8]

In [50]:
df.columns = ['Time','Bid','Ask','Mid','Diretion','Or_time','Or_Bid','Or_Ask']

In [51]:
df_time = df.set_index('Time').copy()

In [52]:
df['Mid'] = 0.5*(df['Bid']+df['Ask'])

In [53]:
df_time['Mid'] = 0.5*(df_time['Bid']+df_time['Ask'])

In [164]:
df_order = df_raw.iloc[0:90,11:16]

#### Assume each order has size of 1, then calculate thePnL
## MT

In [56]:
df_mt = df.copy()

In [57]:
df_mt = df_mt.fillna(0.0)

In [58]:
df_mt['PnL'] = 0.5*(df_mt['Or_Ask'].astype(float)-df_mt['Or_Bid'].astype(float)) 

In [60]:
df_mt['PnL'].sum()

0.006235000000041402

In [62]:
len(df_mt)

431814

#### So, assume each order has size of 1, the excution cost is 0.006235

## OMMside

In [151]:
df_ommside = df.copy()
df_ommside_sb = df_ommside.dropna()
df_ommside = df_ommside.fillna(0.0)
delta = timedelta(days=0,seconds=1)
index_list = df_ommside_sb.index

In [152]:
# c1: Condition, the name of column used; tte is a int; sl is a float (20,0.0003)
def get_pnl_side(df_input,tte,sl):
    pnl = [] # for each row there is a pnl value, it there is no trade, append nan, positive cost lose money
    reason = [] #reason recording
    wait_time = []
    length = len(df_input)
    for i in index_list:
        if df_input['Diretion'][i] == 'B':
            price_aim = df_input['Bid'][i]
            # 3 conditions that leads to aggress the price --- tte,sl and price reaches our aim; put tte in loop's condition
            # ... and within the loop we only need two condition, with break
            for j in range(tte):
                price_temp = df_input['Ask'][i+j+1]
                price_change = df_input['Ask'][i+j+1] - df_input['Ask'][i]
                cost = df_input['Ask'][i+j+1] - df_input['Mid'][i]
                if price_change >= sl:
                    pnl.append(-1*cost)
                    reason.append('sl')
                    wait_time.append(j+1)
                    break;
                if price_temp <= price_aim:
                    pnl.append(-1*cost)
                    reason.append('success')
                    wait_time.append(j+1)
                    break;
                if j == tte-1:
                    pnl.append(-1*cost)
                    reason.append('tte')
                    wait_time.append(j+1)
                    #print(wait_time)
                    
        elif df_input['Diretion'][i] == 'S':
            price_aim = df_input['Ask'][i]
            # 3 conditions that leads to aggress the price --- tte,sl and price reaches our aim; put tte in loop's condition
            # ... and within the loop we only need two condition, with break
            for j in range(tte):
                price_temp = df_input['Bid'][i+j+1]
                price_change = df_input['Bid'][i+j+1] - df_input['Bid'][i]
                cost = df_input['Bid'][i+j+1] - df_input['Mid'][i]
                if -price_change >= sl:
                    pnl.append(1*cost)
                    reason.append('sl')
                    wait_time.append(j+1)
                    break;
                if price_temp >= price_aim:
                    pnl.append(1*cost)
                    reason.append('success')
                    wait_time.append(j+1)
                    break;
                if j == tte-1:
                    pnl.append(1*cost)
                    reason.append('tte')
                    wait_time.append(j+1)
        else:
            pnl.append(np.nan)
            reason.append('none')
            wait_time.append(np.nan)
    result = [pnl,reason,wait_time]
    return pd.DataFrame(result).T

### OMMmid

In [153]:
df_ommmid = df.copy()
df_ommmid_sb = df_ommmid.dropna()
df_ommmid = df_ommside.fillna(0.0)
delta = timedelta(days=0,seconds=1)
index_list = df_ommmid_sb.index

In [154]:
def get_pnl_mid(df_input,tte,sl):
    pnl = [] # for each row there is a pnl value, it there is no trade, append nan, positive cost lose money
    reason = [] #reason recording
    wait_time = []
    length = len(df_input)
    for i in index_list:
        if df_input['Diretion'][i] == 'B':
            price_aim = df_input['Mid'][i]
            # 3 conditions that leads to aggress the price --- tte,sl and price reaches our aim; put tte in loop's condition
            # ... and within the loop we only need two condition, with break
            for j in range(tte):
                price_temp = df_input['Ask'][i+j+1]
                price_change = df_input['Ask'][i+j+1] - df_input['Ask'][i]
                cost = df_input['Ask'][i+j+1] - df_input['Mid'][i]
                if price_change >= sl:
                    pnl.append(-1*cost)
                    reason.append('sl')
                    wait_time.append(j+1)
                    break;
                if price_temp <= price_aim:
                    pnl.append(-1*cost)
                    reason.append('success')
                    wait_time.append(j+1)
                    break;
                if j == tte-1:
                    pnl.append(-1*cost)
                    reason.append('tte')
                    wait_time.append(j+1)
                    #print(wait_time)
                    
        elif df_input['Diretion'][i] == 'S':
            price_aim = df_input['Mid'][i]
            # 3 conditions that leads to aggress the price --- tte,sl and price reaches our aim; put tte in loop's condition
            # ... and within the loop we only need two condition, with break
            for j in range(tte):
                price_temp = df_input['Bid'][i+j+1]
                price_change = df_input['Bid'][i+j+1] - df_input['Bid'][i]
                cost = df_input['Bid'][i+j+1] - df_input['Mid'][i]
                if -price_change >= sl:
                    pnl.append(1*cost)
                    reason.append('sl')
                    wait_time.append(j+1)
                    break;
                if price_temp >= price_aim:
                    pnl.append(1*cost)
                    reason.append('success')
                    wait_time.append(j+1)
                    break;
                if j == tte-1:
                    pnl.append(1*cost)
                    reason.append('tte')
                    wait_time.append(j+1)
        else:
            pnl.append(np.nan)
            reason.append('none')
            wait_time.append(np.nan)
    result = [pnl,reason,wait_time]
    return pd.DataFrame(result).T

In [175]:
def OMM_concat(df1,df2,tte,sl):
    df1 = get_pnl_side(df1,tte,sl)
    df2 = get_pnl_mid(df2,tte,sl)
    #result = pd.concat([df_order,df1,df2],axis=1, sort=False)
    #result.columns = ['Time','Order','Then_Bid','Then_Ask','pnl_side','reason_side','wt_side','pnl_mid','reason_mid','wt_mid']
    result = pd.concat([df1,df2],axis=1, sort=False)
    result.columns = ['pnl_side','reason_side','wt_side','pnl_mid','reason_mid','wt_mid']
    return result

In [177]:
OMM_concat(df_ommside,df_ommmid,20,0.0003)['pnl_side'].mean()

-0.00011594444444393837

In [ ]:
def OMM_stats(df1,df2,tte,sl):
    result_df = OMM_concat(df1,df2,tte,sl)
    mean_side
    mean_mid
    median_side
    median_mid
    
    df = pd.DataFrame([[1,1.23,'Hello']], columns=list('ABC'))

In [170]:
for i in np.arange(11, 17, 0.5).tolist():
    

11.0
11.5
12.0
12.5
13.0
13.5
14.0
14.5
15.0
15.5
16.0
16.5
